In [1]:
%matplotlib widget
import os
os.environ["GDAL_DATA"] = "/home/parndt/anaconda3/envs/geo_py37/share/gdal"
os.environ["PROJ_LIB"] = "/Users/parndt/anaconda3/envs/geo_py37/share/proj"
os.environ["PROJ_DATA"] = "/Users/parndt/anaconda3/envs/geo_py37/share/proj"
import os
import re
import json
import h5py
import math
import shutil
import zipfile
import shapely
import requests
import datetime
import traceback
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry.polygon import orient
from shapely.geometry import Polygon, mapping
from xml.etree import ElementTree as ET
import hdbscan
import matplotlib
import matplotlib.pylab as plt
from matplotlib.patches import Rectangle
from matplotlib.gridspec import GridSpec
from cmcrameri import cm as cmc
from mpl_toolkits.axes_grid1 import make_axes_locatable
from IPython.display import Image, display
from matplotlib.collections import PatchCollection
from sklearn.neighbors import KDTree
from scipy.stats import binned_statistic
from scipy.signal import find_peaks
import rasterio as rio
from rasterio import plot as rioplot
from rasterio import warp
from rasterio import Affine as A
from rasterio.enums import ColorInterp
from rasterio.windows import Window
from rasterio.windows import from_bounds
from rasterio.transform import TransformMethodsMixin
from rasterio.enums import Resampling
from rasterio.warp import reproject
from rasterio.crs import CRS
from scipy.interpolate import RectBivariateSpline
from scipy.interpolate import RegularGridInterpolator
from scipy import signal
from utils import read_atl03
from utils import intersection
from ed.edcreds import getedcreds

def download_is2(short_name='ATL03', start_date='2018-01-01', end_date='2030-01-01', uid='userid', pwd='pwd', 
                 bbox=None, shape=None, vars_sub='all', output_dir='nsidc_outputs'):
    
    bounding_box = '%.7f,%.7f,%.7f,%.7f' % tuple(boundbox)

    start_time = '00:00:00'
    end_time = '23:59:59'
    temporal = start_date + 'T' + start_time + 'Z' + ',' + end_date + 'T' + end_time + 'Z'

    cmr_collections_url = 'https://cmr.earthdata.nasa.gov/search/collections.json'
    granule_search_url = 'https://cmr.earthdata.nasa.gov/search/granules'
    base_url = 'https://n5eil02u.ecs.nsidc.org/egi/request'

    # Get json response from CMR collection metadata
    params = {'short_name': short_name}
    response = requests.get(cmr_collections_url, params=params)
    results = json.loads(response.content)

    # Find all instances of 'version_id' in metadata and print most recent version number
    versions = [el['version_id'] for el in results['feed']['entry']]
    latest_version = max(versions)
    capability_url = f'https://n5eil02u.ecs.nsidc.org/egi/capabilities/{short_name}.{latest_version}.xml'

    search_params = {'short_name': short_name, 'version': latest_version, 'temporal': temporal, 'page_size': 100, 'page_num': 1}
    if boundbox is not None: search_params['bounding_box'] = bounding_box
    elif shape is not None: search_params['polygon'] = polygon
    else: print('No spatial filtering criteria were given.')

    headers= {'Accept': 'application/json'}
    print("Search parameters:", search_params)

    # query for granules 
    granules = []
    headers={'Accept': 'application/json'}
    while True:
        response = requests.get(granule_search_url, params=search_params, headers=headers)
        results = json.loads(response.content)
        if len(results['feed']['entry']) == 0: break
        granules.extend(results['feed']['entry'])
        search_params['page_num'] += 1

    print('\nFound %i %s version %s granules over the search area between %s and %s.' % (len(granules), short_name, latest_version, 
                                                                              start_date, end_date))
    if len(granules) == 0: print('None')
    for result in granules:
        print('  '+result['producer_granule_id'], f', {float(result["granule_size"]):.2f} MB',sep='')

    if shape is not None:
        gdf = gpd.read_file(geojson_filepath)

        # Simplify polygon for complex shapes in order to pass a reasonable request length to CMR. 
        # The larger the tolerance value, the more simplified the polygon.
        # Orient counter-clockwise: CMR polygon points need to be provided in counter-clockwise order. 
        # The last point should match the first point to close the polygon.
        poly = orient(gdf.simplify(0.05, preserve_topology=False).loc[0],sign=1.0)

        geojson_data = gpd.GeoSeries(poly).to_json() # Convert to geojson
        geojson_data = geojson_data.replace(' ', '') #remove spaces for API call

        #Format dictionary to polygon coordinate pairs for CMR polygon filtering
        polygon = ','.join([str(c) for xy in zip(*poly.exterior.coords.xy) for c in xy])

        print('\nInput geojson:', geojson)
        print('Simplified polygon coordinates based on geojson input:', polygon)

    # Create session to store cookie and pass credentials to capabilities url
    session = requests.session()
    s = session.get(capability_url)
    response = session.get(s.url,auth=(uid,pwd))

    root = ET.fromstring(response.content)

    #collect lists with each service option
    subagent = [subset_agent.attrib for subset_agent in root.iter('SubsetAgent')]

    # this is for getting possible variable values from the granule search
    if len(subagent) > 0 :
        # variable subsetting
        variables = [SubsetVariable.attrib for SubsetVariable in root.iter('SubsetVariable')]  
        variables_raw = [variables[i]['value'] for i in range(len(variables))]
        variables_join = [''.join(('/',v)) if v.startswith('/') == False else v for v in variables_raw] 
        variable_vals = [v.replace(':', '/') for v in variables_join]

    # make sure to only request the variables that are available
    def intersection(lst1, lst2):
        lst3 = [value for value in lst1 if value in lst2]
        return lst3
    if vars_sub == 'all':
        var_list_subsetting = ''
    else:
        var_list_subsetting = intersection(variable_vals,var_list)

    if len(subagent) < 1 :
        print('No services exist for', short_name, 'version', latest_version)
        agent = 'NO'
        coverage,Boundingshape = '',''
    else:
        agent = ''
        subdict = subagent[0]
        if subdict['spatialSubsettingShapefile'] == 'true':
            if boundbox is not None:
                Boundingshape, polygon, bbox = '', '', bounding_box
            if shape is not None:
                Boundingshape, bbox = '', geojson_data
            else:
                Boundingshape = ''
        coverage = ','.join(var_list_subsetting)

    #Set the request mode to asynchronous if the number of granules is over 100, otherwise synchronous is enabled by default
    if len(granules) > 100:
        request_mode = 'async'
        page_size = 2000
    else: 
        page_size = 100
        request_mode = 'stream'
    #Determine number of orders needed for requests over 2000 granules. 
    page_num = math.ceil(len(granules)/page_size)

    print('  --> There will be', page_num, 'total order(s) processed for our', short_name, 'request.')
    param_dict = {'short_name': short_name, 
                  'version': latest_version, 
                  'temporal': temporal, 
                  'bbox': bbox,
                  'bounding_box': bounding_box,
                  'Boundingshape': Boundingshape, 
                  'polygon': polygon,
                  'Coverage': coverage, 
                  'page_size': page_size, 
                  'request_mode': request_mode, 
                  'agent': agent, 
                  'email': email, }

    #Remove blank key-value-pairs
    param_dict = {k: v for k, v in param_dict.items() if v != ''}

    #Convert to string
    param_string = '&'.join("{!s}={!r}".format(k,v) for (k,v) in param_dict.items())
    param_string = param_string.replace("'","")

    #Print API base URL + request parameters
    endpoint_list = [] 
    for i in range(page_num):
        page_val = i + 1
        API_request = api_request = f'{base_url}?{param_string}&page_num={page_val}'
        endpoint_list.append(API_request)

    print('\n', *endpoint_list, sep = "\n") 

    # Create an output folder if the folder does not already exist.
    path = str(os.getcwd() + '/' + output_dir)
    if not os.path.exists(path):
        os.mkdir(path)

    # Different access methods depending on request mode:
    if request_mode=='async':
        # Request data service for each page number, and unzip outputs
        for i in range(page_num):
            page_val = i + 1
            print('Order: ', page_val)

        # For all requests other than spatial file upload, use get function
            param_dict['page_num'] = page_val
            request = session.get(base_url, params=param_dict)

            print('Request HTTP response: ', request.status_code)

        # Raise bad request: Loop will stop for bad response code.
            request.raise_for_status()
            print('Order request URL: ', request.url)
            esir_root = ET.fromstring(request.content)
            print('Order request response XML content: ', request.content)

        #Look up order ID
            orderlist = []   
            for order in esir_root.findall("./order/"):
                orderlist.append(order.text)
            orderID = orderlist[0]
            print('order ID: ', orderID)

        #Create status URL
            statusURL = base_url + '/' + orderID
            print('status URL: ', statusURL)

        #Find order status
            request_response = session.get(statusURL)    
            print('HTTP response from order response URL: ', request_response.status_code)

        # Raise bad request: Loop will stop for bad response code.
            request_response.raise_for_status()
            request_root = ET.fromstring(request_response.content)
            statuslist = []
            for status in request_root.findall("./requestStatus/"):
                statuslist.append(status.text)
            status = statuslist[0]
            print('Data request ', page_val, ' is submitting...')
            print('Initial request status is ', status)

        #Continue loop while request is still processing
            while status == 'pending' or status == 'processing': 
                print('Status is not complete. Trying again.')
                time.sleep(10)
                loop_response = session.get(statusURL)

        # Raise bad request: Loop will stop for bad response code.
                loop_response.raise_for_status()
                loop_root = ET.fromstring(loop_response.content)

        #find status
                statuslist = []
                for status in loop_root.findall("./requestStatus/"):
                    statuslist.append(status.text)
                status = statuslist[0]
                print('Retry request status is: ', status)
                if status == 'pending' or status == 'processing':
                    continue

        #Order can either complete, complete_with_errors, or fail:
        # Provide complete_with_errors error message:
            if status == 'complete_with_errors' or status == 'failed':
                messagelist = []
                for message in loop_root.findall("./processInfo/"):
                    messagelist.append(message.text)
                print('error messages:')
                pprint.pprint(messagelist)

        # Download zipped order if status is complete or complete_with_errors
            if status == 'complete' or status == 'complete_with_errors':
                downloadURL = 'https://n5eil02u.ecs.nsidc.org/esir/' + orderID + '.zip'
                print('Zip download URL: ', downloadURL)
                print('Beginning download of zipped output...')
                zip_response = session.get(downloadURL)
                # Raise bad request: Loop will stop for bad response code.
                zip_response.raise_for_status()
                with zipfile.ZipFile(io.BytesIO(zip_response.content)) as z:
                    z.extractall(path)
                print('Data request', page_val, 'is complete.')
            else: print('Request failed.')

    else:
        for i in range(page_num):
            page_val = i + 1
            print('\nOrder: ', page_val)
            print('Requesting...')
            request = session.get(base_url, params=param_dict)
            print('HTTP response from order response URL: ', request.status_code)
            request.raise_for_status()
            d = request.headers['content-disposition']
            fname = re.findall('filename=(.+)', d)
            dirname = os.path.join(path,fname[0].strip('\"'))
            print('Downloading...')
            open(dirname, 'wb').write(request.content)
            print('Data request', page_val, 'is complete.')

        # Unzip outputs
        for z in os.listdir(path): 
            if z.endswith('.zip'): 
                zip_name = path + "/" + z 
                zip_ref = zipfile.ZipFile(zip_name) 
                zip_ref.extractall(path) 
                zip_ref.close() 
                os.remove(zip_name)

    # Clean up Outputs folder by removing individual granule folders 
    for root, dirs, files in os.walk(path, topdown=False):
        for file in files:
            try:
                shutil.move(os.path.join(root, file), path)
            except OSError:
                pass
        for name in dirs:
            os.rmdir(os.path.join(root, name))    

In [9]:
# for San G Veg offpointing track 82 2021-09-27
dtm_bbox = [[520925.877649,525919.746883],[3766456.711576,3773181.723345]]
coords_latlon = warp.transform({'init': 'epsg:32611'}, {'init': 'epsg:4326'}, dtm_bbox[0], dtm_bbox[1])
boundbox = list(np.array(coords_latlon).T.flatten())
start_date = '2021-09-27'
end_date = '2021-09-27'
output_dir = 'data/IS2/sang_test_veg_offpointing'
uid, pwd, email = getedcreds()
download_is2(start_date=start_date, end_date=end_date, uid=uid, pwd=pwd, bbox=boundbox, output_dir=output_dir)

Search parameters: {'short_name': 'ATL03', 'version': '005', 'temporal': '2021-09-27T00:00:00Z,2021-09-27T23:59:59Z', 'page_size': 100, 'page_num': 1, 'bounding_box': '-116.7732992,34.0385789,-116.7189984,34.0991181'}

Found 2 ATL03 version 005 granules over the search area between 2021-09-27 and 2021-09-27.
  ATL03_20210927194224_00821302_005_01.h5, 5508.05 MB
  ATL03_20210927194224_00821302_005_01.h5, 5508.05 MB
  --> There will be 1 total order(s) processed for our ATL03 request.


https://n5eil02u.ecs.nsidc.org/egi/request?short_name=ATL03&version=005&temporal=2021-09-27T00:00:00Z,2021-09-27T23:59:59Z&bbox=-116.7732992,34.0385789,-116.7189984,34.0991181&bounding_box=-116.7732992,34.0385789,-116.7189984,34.0991181&page_size=100&request_mode=stream&email=philipp.arndt@aya.yale.edu&page_num=1

Order:  1
Requesting...
HTTP response from order response URL:  200
Downloading...
Data request 1 is complete.


In [10]:
coords_dtm = warp.transform({'init': 'epsg:4326'},{'init': 'epsg:32611'}, [boundbox[0], boundbox[2]],[boundbox[1],boundbox[3]])
coords_dtm

([520925.8776490004, 525919.7468830011],
 [3766456.7115759994, 3773181.7233450003])